In [ ]:
# Run if you are executing this notebook in Colab to ensure tf v2.x.x
!pip install -U tensorflow keras

In [1]:
import datetime
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns
from keras import models, layers, datasets
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [ ]:
plt.rc("font", size=14)
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

# A SF Permits Cleaning

### Methode um Y/NaN Spalten zu konvertieren

In [ ]:
def replace_y_with_0_1(df: pd.DataFrame, column: str):
    """
    Ersetzt Y und leere Felder in Spalten die nur Y und leere Felder erhalten durch 1 und 0 
    :param df: Dataframe in dem sich die Spalten befinden
    :param column: Name der Spalte in der die Werte ersetzt werden sollen
    :return: DataFrame mit modifizierten Spalten
    """
    df[column].fillna(0, inplace=True)
    df[column].replace('Y', 1, inplace=True)

### Datenset in ein Dataframe laden laden und Vorschau anzeigen

In [ ]:
sanfrancisco_df = pd.read_csv("./Building_Permits.csv")

# sanfrancisco_df.head(5)
sanfrancisco_df.profile_report()

### Fehlende Werte kontrollieren

Um einen Überblick über die Daten zu bekommen, lassen wir uns ausgeben wie hoch der Anteil an fehlenden Werten ist.

In [ ]:
round(sanfrancisco_df.isnull().sum() / len(sanfrancisco_df) * 100, 2)

Und als nächstes schauen wir uns die Werte die in den einzelnen Spalten sind an.  
Format:
- Alle möglichen Werte: zuerst der Werte dann die Anzahl (wenn es zu viele werte gibt wird abgekürzt)
- als letzte Zeile der Name der Spalte und die Menge an einzigartigen Werte

In [20]:
for key in sanfrancisco_df.keys():
    vals = sanfrancisco_df[key].value_counts(dropna=False)
    print(vals)
    print("-----------------------------------------------------------")

201602179765    101
201602179758     66
201602179775     30
201409166451      9
201708165004      9
               ... 
M861647           1
201308194693      1
M688867           1
201506108677      1
201711224750      1
Name: Permit Number, Length: 181495, dtype: int64
-----------------------------------------------------------
8    178844
3     14663
4      2892
2       950
6       600
7       511
1       349
5        91
Name: Permit Type, dtype: int64
-----------------------------------------------------------
otc alterations permit                 178844
additions alterations or repairs        14663
sign - erect                             2892
new construction wood frame               950
demolitions                               600
wall or painted sign                      511
new construction                          349
grade or quarry or fill or excavate        91
Name: Permit Type Definition, dtype: int64
-----------------------------------------------------------
09/15/2017 

Wie man in unserer Ausgabe oben sehen kann, gibt es einige Spalten in denen nur `Y/NaN` Werte vorhanden sind. `Y` steht
dabei für true und `NaN` Feld für false. Um das für unsere Vorhersage besser verwenden zu können ersetzen wir das
`Y` durch eine `1` und füllen die leeren Zellen mit `0` auf.  

- Structural Notification
- Voluntary Soft-Story Retrofit
- Fire Only Permit
- Site Permit

Die Spalte `TIDF Compliance` ist vermutlich auch eine solche Spalte, hat aber insgesamt nur zwei einzelne Werte und ist
somit uninteressant für uns und wird entfernt.

In [ ]:
replace_y_with_0_1(sanfrancisco_df, "Structural Notification")
replace_y_with_0_1(sanfrancisco_df, "Voluntary Soft-Story Retrofit")
replace_y_with_0_1(sanfrancisco_df, "Fire Only Permit")
replace_y_with_0_1(sanfrancisco_df, "Site Permit")

sanfrancisco_df.drop("TIDF Compliance", axis='columns', inplace=True)

Da wir mit dem `Supervisor District` bereits eine geografische Einteilung in Regionen haben, entfernen wir alle
Adressdaten und die Koordinaten, da wir die für unsere Vorhersage in Textform nicht gebrauchen können und
One-Hot-Encoding keinen Sinn macht:
- Block
- Lot
- Street Number
- Street Number Suffix
- Street Name
- Street Suffix
- Unit
- Unit Suffix
- Neighborhoods - Analysis Boundaries
- Zipcode
- Location

Außerdem entfernen wir Felder die Freitext oder IDs enthalten:
- Description
- Existing Use
- Proposed Use
- Permit Number
- Record ID

In [ ]:
sanfrancisco_df.drop("Block", axis=1, inplace=True)
sanfrancisco_df.drop("Lot", axis=1, inplace=True)
sanfrancisco_df.drop("Street Number", axis=1, inplace=True)
sanfrancisco_df.drop("Street Number Suffix", axis=1, inplace=True)
sanfrancisco_df.drop("Street Name", axis=1, inplace=True)
sanfrancisco_df.drop("Street Suffix", axis=1, inplace=True)
sanfrancisco_df.drop("Unit", axis=1, inplace=True)
sanfrancisco_df.drop("Unit Suffix", axis=1, inplace=True)
sanfrancisco_df.drop("Neighborhoods - Analysis Boundaries", axis=1, inplace=True)
sanfrancisco_df.drop("Zipcode", axis=1, inplace=True)
sanfrancisco_df.drop("Location", axis=1, inplace=True)

sanfrancisco_df.drop("Description", axis=1, inplace=True)
sanfrancisco_df.drop("Existing Use", axis=1, inplace=True)
sanfrancisco_df.drop("Proposed Use", axis=1, inplace=True)
sanfrancisco_df.drop("Permit Number", axis=1, inplace=True)
sanfrancisco_df.drop("Record ID", axis=1, inplace=True)

Und alle Zeilen die keinen `Supervisor District` haben werden entfernt da wir die Zeilen nicht manuell den Distrikten
zuordnen können.

In [ ]:
sanfrancisco_df = sanfrancisco_df[sanfrancisco_df["Supervisor District"].notna()]

Bei den folgenden Spalten ersetzen wir leere Felder durch `0` da wir uns nur die Differenz zwischen vorher/nachher
anschauen wollen und das somit keinen Einfluss hat.
- Number of Existing Stories
- Number of Proposed Stories
- Existing Units
- Proposed Units

In [ ]:
sanfrancisco_df["Number of Existing Stories"].fillna(0, inplace=True)
sanfrancisco_df["Number of Proposed Stories"].fillna(0, inplace=True)
sanfrancisco_df["Existing Units"].fillna(0, inplace=True)
sanfrancisco_df["Proposed Units"].fillna(0, inplace=True)

Die folgenden Spalten sind redundant zu ihren `XYZ Description` Spalten und werden somit nicht weiter benötigt.
- Existing Construction Type
- Proposed Construction Type

Zusätzlich werden die leeren Felder der anderen Spalten mit `no constr type 0`gefüllt, weil wir keine sinnvolle Annahme
über den tatsächlichen Wert machen können und so die aufgefüllten Spalten erkennen können.

In [ ]:
sanfrancisco_df.drop("Existing Construction Type", axis=1, inplace=True)
sanfrancisco_df.drop("Proposed Construction Type", axis=1, inplace=True)
sanfrancisco_df["Existing Construction Type Description"].fillna("no constr type 0", inplace=True)
sanfrancisco_df["Proposed Construction Type Description"].fillna("no constr type 0", inplace=True)

In [ ]:
sanfrancisco_df.isnull().sum()

In den folgenden Spalten füllen wir die leeren Felder mit dem Median der Spalte:
- Estimated Cost
- Revised Cost
- Plansets

In [ ]:
def replace_with_median(df: pd.DataFrame, column: str):
    median = df[column].median(skipna=True)
    df[column].fillna(median, inplace=True)

In [ ]:
replace_with_median(sanfrancisco_df, "Estimated Cost")
replace_with_median(sanfrancisco_df, "Revised Cost")
replace_with_median(sanfrancisco_df, "Plansets")

In [ ]:
sanfrancisco_df.isnull().sum()

Die restlichen Spalten mit fehlenden Spalten sind Spalten mit Daten. Einige sind nicht ausgefüllt weil sie erst in der
Zukunft gefüllt werden können, wir füllen sie mit `00/00/0000` um später entsprechend die Werte auswählen zu können.

In [ ]:
sanfrancisco_df["Issued Date"].fillna("00/00/0000", inplace=True)
sanfrancisco_df["Completed Date"].fillna("00/00/0000", inplace=True)
sanfrancisco_df["First Construction Document Date"].fillna("00/00/0000", inplace=True)
sanfrancisco_df["Permit Expiration Date"].fillna("00/00/0000", inplace=True)

In [ ]:
sanfrancisco_df.isnull().sum()

# B SF Permits Exploration

### Lineare abhängikeiten suchen 
Um eine Idee davon zu bekommen wie stark die einzelnen Spalten von einander abhängen berrechnen wir eine
Korrelationsmatrix. Je näher die Werte an 1/-1 liegen desto stärker ist die Abhängigkeit.

In [ ]:
correlation_matrix = sanfrancisco_df.corr().round(2)
sns.set(rc={'figure.figsize':(12, 9)})
sns.heatmap(data=correlation_matrix, annot=True)

# C SF Permits Prediction

### Methode für One-Hot-Encoding

In [ ]:
def one_hot_encode(df: pd.DataFrame, col: str) -> pd.DataFrame:
    """
    Erzeugt Dummy-Spalten für jeden Wert in der Quell-Spalte.
    
    Beispiel:
    Wenn eine Tabelle die Spalte 'Klasse' mit 0, 1, 2 als Werte hat, werden die Spalte Klasse 0, Klasse 1 und Klasse 2
    erstellt und die Zeilen der Tabelle an den entsprechenden Stellen auf 0 bzw. 1 gesetzt. 
    :param df: Dataframe in dem die zu ersetzende Spalte ist
    :param col: Spalte die One-Hot-Encoded werden soll
    :return: modifizierter DataFrame
    """
    return df.assign(**{str(col + " " + str(val)): [1 if str(val) in str(cell) else 0 for cell in df[col]] 
                                   for val in df[col].unique()})

def days_between_dates(d1: str, d2: str) -> int:
    """
    Akzeptiert zwei Daten in der Form DD/MM/YYYY und berechnet die Tage zwischen den beiden Daten.
    :param d1: Erstes Datum
    :param d2: Zweites Datum
    :return: Anzahl der Tage zwischen den beiden Daten
    """
    d1 = d1.split("/")
    d2 = d2.split("/")
    if len(d1) != 3 or len(d2) != 3:
        raise ValueError("Date must consist of three numbers divided by '/'")
    d1 = datetime.date(int(d1[2]), int(d1[1]), int(d1[0]))
    d2 = datetime.date(int(d2[2]), int(d2[1]), int(d2[0]))
    return (d2 - d1).days

# D SF Challenges

# E Neural Networks XOR

![XOR Graph](https://github.com/maxmoehl/ATIT2_assignments/blob/master/Assignment_2/XOR.png?raw=1)

w: weight  
b: bias  
t: threshold

Mathematische Funktion:
```
SIGMOID(x) = 1 / (1 + e^(-x))
AND(x, y) = SIGMOID(50x + 50y - 75)
OR(x, y) = SIGMOID(50x + 50y - 25)
NOT(x) = SIGMOID(-50x + 10)

XOR(x, y) = AND(NOT(AND(x, y)), OR(x, y))

XOR(x,y) = SIGMOID(SIGMOID(SIGMOID(50x + 50y - 75) * (-50) + 10) * 50 + SIGMOID(50x + 50y - 25) * 50 - 75)
           \       \       \---------AND---------/             /        \                     /          /
            \       \-------------------NOT-------------------/          \--------OR---------/          /
             \-------------------------------------------AND-------------------------------------------/
```

### Funktionen in Python abbilden und testen

In [ ]:
def SIGMOID(x):
    return 1 / (1 + (math.e ** (-x)))

def AND(x, y):
    return SIGMOID(x * 50 + y * 50 - 75)

def OR(x, y):
    return SIGMOID(x * 50 + y * 50 - 25)
    
def NOT(x):
    return SIGMOID(x * (-50) + 10)

def XOR(x, y):
    return AND(NOT(AND(x, y)),OR(x, y))

print(round(XOR(0, 0), 2))
print(round(XOR(0, 1), 2))
print(round(XOR(1, 0), 2))
print(round(XOR(1, 1), 2))

# F Neural Networks Overfitting

## Daten laden und splitten

Das größte Problem wenn man ein Modell hat das overfitted ist das man zu wenige Daten zum trainieren hat, um das hier zu simulieren verwenden wir nicht die 50.000 Datensätze zum trainieren sondern nur die 10.000.

In [ ]:
(X_test, Y_test), (X_train, Y_train) = datasets.cifar10.load_data()

X_train, X_test = X_train / 255.0, X_test / 255.0

num_classes = 10

## Modell erstellen

Damit das Modell overfitted nehmen wir nur zwei Convolutional und ein MaxPooling Layer und zum auswerten auch nur ein Dense Layer.

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Modell trainieren

Zusätzlich trainieren wir das Modell in 20 Epochen, das Modell wird sich also viel zu sehr an die 10.000 Datensätze die wir verwenden anpassen.

In [ ]:
history = model.fit(X_train, Y_train, batch_size=100, epochs=20, validation_data=(X_test, Y_test))

## Methode zum visualisieren der Ergebnisse

In [ ]:
def plot_results(history, epoch_lim=20):
  plt.plot(history.history['loss'], label='train')
  plt.plot(history.history['val_loss'], label='test')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.ylim([0, 2.5])
  plt.xlim([0, epoch_lim])
  plt.legend(loc='upper left')

## Ergebnisse visualisieren

In [ ]:
print("Train accuracy: {}%".format(round(history.history['accuracy'][-1] * 100, 2)))
print("Test accuracy: {}%".format(round(history.history['val_accuracy'][-1] * 100, 2)))
plot_results(history)

Wie man anhand des Graphen sehr gut sehen kann, steigt der Loss der Testdaten nach ca. 12 Epochen wieder an: das Modell overfitted. Das kann man auch an der Accuracy sehen, für die Trainingsdaten liegt die bei über 90% währrend die Testdaten nur um die 55% haben.

# G Neural Networks Overfitting

## Daten laden und splitten

Um jetzt in diesem Modell das Overfitting zu verhindern verwenden wir die 50.000 Datensätze zum lernen.

In [ ]:
(X_train, Y_train), (X_test, Y_test) = datasets.cifar10.load_data()

X_train, X_test = X_train / 255.0, X_test / 255.0

## Modell erstellen

Damit das Modell die Muster in den Bildern besser erkennen kann fügen wir im Vergleich zu dem ersten Modell eine weitere Conv/MaxPooling Kombi und ein weiteres Dense Layer hinzu.

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Modell trainieren

Um Overfitting zu verhindern verringern wir die Anzahl an Epochen.

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
history = model.fit(X_train, Y_train, batch_size=10, epochs=15, validation_data=(X_test, Y_test), callbacks=[early_stopping])

## Ergebnisse visualisieren

In [ ]:
print("Train accuracy: {}%".format(round(history.history['accuracy'][-1] * 100, 2)))
print("Test accuracy: {}%".format(round(history.history['val_accuracy'][-1] * 100, 2)))
plot_results(history, epoch_lim=15)

Wie man an dem Graphen sehen kann sinkt der Loss sowohl für die Trainingsdaten als auch für die Testdaten kontinuierlich. Das Modell generalisiert wesentlich besser anstatt einfach nur die Bilder aus dem Trainingsset auswendig zu lernen. Der Early-Stop Callback verhindert, dass das Modell durch zu viele Trainingsepochen overfitted.  
Die Accuracy von Train und Test liegen auch sehr nah beieinander.

# H Feedback
### Wieviel Zeit habt ihr mit dem Assignment verbracht?
Max: 15h
### Wie oft habt ihr euch getroffen?
3
### Welche Aufgabe hat euch am besten gefallen?
### Welche Aufgabe hat euch am wenigsten gefallen?